# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2b1c8fc880>
├── 'a' --> tensor([[-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305]])
└── 'x' --> <FastTreeValue 0x7f2b1c8fcfa0>
    └── 'c' --> tensor([[ 0.7541, -1.7117,  1.5779,  0.6308],
                        [ 0.0764, -1.1738,  0.0334,  0.2999],
                        [ 0.2222,  0.7057, -0.2765,  1.3476]])

In [4]:
t.a

tensor([[-1.0470, -0.2211, -0.2071],
        [ 2.0724,  1.4919, -0.7305]])

In [5]:
%timeit t.a

67.2 ns ± 0.088 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2b1c8fc880>
├── 'a' --> tensor([[ 1.4961,  0.6228, -2.0567],
│                   [ 0.2532,  0.2260,  0.3338]])
└── 'x' --> <FastTreeValue 0x7f2b1c8fcfa0>
    └── 'c' --> tensor([[ 0.7541, -1.7117,  1.5779,  0.6308],
                        [ 0.0764, -1.1738,  0.0334,  0.2999],
                        [ 0.2222,  0.7057, -0.2765,  1.3476]])

In [7]:
%timeit t.a = new_value

68.1 ns ± 0.11 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305]]),
    x: Batch(
           c: tensor([[ 0.7541, -1.7117,  1.5779,  0.6308],
                      [ 0.0764, -1.1738,  0.0334,  0.2999],
                      [ 0.2222,  0.7057, -0.2765,  1.3476]]),
       ),
)

In [10]:
b.a

tensor([[-1.0470, -0.2211, -0.2071],
        [ 2.0724,  1.4919, -0.7305]])

In [11]:
%timeit b.a

62.1 ns ± 0.0165 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7030, -0.7632, -1.6972],
               [ 1.2143,  0.3952, -0.7108]]),
    x: Batch(
           c: tensor([[ 0.7541, -1.7117,  1.5779,  0.6308],
                      [ 0.0764, -1.1738,  0.0334,  0.2999],
                      [ 0.2222,  0.7057, -0.2765,  1.3476]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.121 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

850 ns ± 0.0647 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 565 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 486 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2a6d5949a0>
├── 'a' --> tensor([[[-1.0470, -0.2211, -0.2071],
│                    [ 2.0724,  1.4919, -0.7305]],
│           
│                   [[-1.0470, -0.2211, -0.2071],
│                    [ 2.0724,  1.4919, -0.7305]],
│           
│                   [[-1.0470, -0.2211, -0.2071],
│                    [ 2.0724,  1.4919, -0.7305]],
│           
│                   [[-1.0470, -0.2211, -0.2071],
│                    [ 2.0724,  1.4919, -0.7305]],
│           
│                   [[-1.0470, -0.2211, -0.2071],
│                    [ 2.0724,  1.4919, -0.7305]],
│           
│                   [[-1.0470, -0.2211, -0.2071],
│                    [ 2.0724,  1.4919, -0.7305]],
│           
│                   [[-1.0470, -0.2211, -0.2071],
│                    [ 2.0724,  1.4919, -0.7305]],
│           
│                   [[-1.0470, -0.2211, -0.2071],
│                    [ 2.0724,  1.4919, -0.7305]]])
└── 'x' --> <FastTreeValue 0x7f2a6d594940>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 75.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2a7419baf0>
├── 'a' --> tensor([[-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305],
│                   [-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305],
│                   [-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305],
│                   [-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305],
│                   [-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305],
│                   [-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305],
│                   [-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305],
│                   [-1.0470, -0.2211, -0.2071],
│                   [ 2.0724,  1.4919, -0.7305]])
└── 'x' --> <FastTreeValue 0x7f2b18871ee0>
    └── 'c' --> tensor([[ 0.7541, -1.7117,  1.5779,  0.6308],
                        [ 0.0764, -1.1738,  0.0334,  0.2999],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 98.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.0470, -0.2211, -0.2071],
                [ 2.0724,  1.4919, -0.7305]],
       
               [[-1.0470, -0.2211, -0.2071],
                [ 2.0724,  1.4919, -0.7305]],
       
               [[-1.0470, -0.2211, -0.2071],
                [ 2.0724,  1.4919, -0.7305]],
       
               [[-1.0470, -0.2211, -0.2071],
                [ 2.0724,  1.4919, -0.7305]],
       
               [[-1.0470, -0.2211, -0.2071],
                [ 2.0724,  1.4919, -0.7305]],
       
               [[-1.0470, -0.2211, -0.2071],
                [ 2.0724,  1.4919, -0.7305]],
       
               [[-1.0470, -0.2211, -0.2071],
                [ 2.0724,  1.4919, -0.7305]],
       
               [[-1.0470, -0.2211, -0.2071],
                [ 2.0724,  1.4919, -0.7305]]]),
    x: Batch(
           c: tensor([[[ 0.7541, -1.7117,  1.5779,  0.6308],
                       [ 0.0764, -1.1738,  0.0334,  0.2999],
                       [ 0.2222,  0.7057, -0.2765,  1.3476]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305],
               [-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305],
               [-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305],
               [-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305],
               [-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305],
               [-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305],
               [-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305],
               [-1.0470, -0.2211, -0.2071],
               [ 2.0724,  1.4919, -0.7305]]),
    x: Batch(
           c: tensor([[ 0.7541, -1.7117,  1.5779,  0.6308],
                      [ 0.0764, -1.1738,  0.0334,  0.2999],
                      [ 0.2222,  0.7057, -0.2765,  1.3476],
                      [ 0.7541, -1.7117,  1.5779,  0.6308],
                      [ 0.0764, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 953 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
